In [2]:
import os
import pickle
import numpy as np
from glob import glob
from scfw.frank_wolfe import run_frank_wolfe
from scopt.prox_newton import run_prox_newton
from scopt.prox_grad import run_prox_grad
from problems.portfolio import PortfolioProblem
from problems.poisson import PoissonProblem
from problems.kl import KLProblem
from problems.dwd import DwdProblem
from problems.cov_estimation import CovEstProblem

## Poisson

In [3]:
policy_list = ['standard', 'sc', 'line_search', 'backtracking']
files = ['a1a', 'a2a', 'a3a', 'a4a']
for file_name in files:
    poisson_path = os.path.join('./data', file_name)
    poisson_problem = PoissonProblem(path=poisson_path)
    poisson_results = {}
    print(file_name)
    for policy in policy_list:
        _, _, _, f_hist, time_hist = run_frank_wolfe(
                                        poisson_problem,
                                        alpha_policy=policy,
                                        max_iter=50000,
                                        print_every=20000
                                        )
        poisson_results[policy] = {'f_hist': f_hist, 'time_hist': time_hist}
    
    _, _, f_hist, time_hist = run_prox_newton(
                                        poisson_problem,
                                        use_two_phase=True,
                                        Lest='estimate',
                                        print_every=10
                                        )
    poisson_results['prox_newton'] = {'f_hist': f_hist, 'time_hist': time_hist}
    _, _, f_hist, time_hist = run_prox_grad(
                                        poisson_problem,
                                        max_iter=50000,
                                        bb_type=2,
                                        print_every=20000
                                        )
    poisson_results['prox_grad'] = {'f_hist': f_hist, 'time_hist': time_hist}
    result_file_path = os.path.join('./results/poisson', file_name + '.pkl')
    with open(result_file_path, 'wb') as f:
        pickle.dump(poisson_results, f)

a1a
iter=1, stepsize=6.67e-01, criterion=1.00e+00, upper_bound=3.66e+03, lower_bound=-2.10e+06, real_Gap=2.10e+06, f_val=3658.7395315749823
iter=20000, stepsize=1.00e-04, criterion=6.53e-05, upper_bound=1.64e+03, lower_bound=1.64e+03, real_Gap=8.14e-01, f_val=1640.2066354183783
iter=40000, stepsize=5.00e-05, criterion=3.71e-05, upper_bound=1.64e+03, lower_bound=1.64e+03, real_Gap=8.01e-01, f_val=1640.2360582031276
8.333545923233032
iter=1, stepsize=1.59e-05, criterion=2.90e-03, upper_bound=3.66e+03, lower_bound=-2.10e+06, real_Gap=2.10e+06, f_val=3658.7395315749823
iter=20000, stepsize=1.16e-07, criterion=1.74e-06, upper_bound=1.64e+03, lower_bound=1.64e+03, real_Gap=3.38e-01, f_val=1640.4753347660203
iter=40000, stepsize=4.05e-07, criterion=9.41e-07, upper_bound=1.64e+03, lower_bound=1.64e+03, real_Gap=1.66e-01, f_val=1640.3133549052823
10.036835432052612
iter=1, stepsize=4.77e-03, criterion=8.71e-01, upper_bound=3.66e+03, lower_bound=-2.10e+06, real_Gap=2.10e+06, f_val=3658.739531574

## Portfolio

In [2]:
portfolio_results = {}
policy_list = ['standard', 'lloo', 'sc', 'line_search', 'backtracking']
files = ['syn_1000_800_10_50', 'syn_1000_1200_10_50', 'syn_1000_1500_10_50',\
    'syn_1000_800_10_50_1', 'syn_1000_1200_10_50_1', 'syn_1000_1500_10_50_1',\
        'syn_1000_800_10_50_2', 'syn_1000_1200_10_50_2', 'syn_1000_1500_10_50_2',\
            'syn_1000_800_10_50_3', 'syn_1000_1200_10_50_3', 'syn_1000_1500_10_50_3']
for file_name in files:
    portfolio_path = os.path.join('./data', file_name + '.mat')
    portfolio_problem = PortfolioProblem(path=portfolio_path, llo_oracle='lloo_simplex')
    portfolio_results = {}
    print(file_name)
    for policy in policy_list:
        _, _, _, f_hist, time_hist = run_frank_wolfe(
                                        portfolio_problem,
                                        alpha_policy=policy,
                                        max_iter=50000,
                                        print_every=20000
                                        )
        portfolio_results[policy] = {'f_hist': f_hist, 'time_hist': time_hist}
    
    _, _, f_hist, time_hist = run_prox_newton(
                                        portfolio_problem,
                                        use_two_phase=False,
                                        Lest='backtracking',
                                        print_every=10
                                        )
    portfolio_results['scopt'] = {'f_hist': f_hist, 'time_hist': time_hist}
    _, _, f_hist, time_hist = run_prox_grad(
                                        portfolio_problem,
                                        max_iter=50000,
                                        bb_type=2,
                                        print_every=10000
                                        )
    portfolio_results['prox_grad'] = {'f_hist': f_hist, 'time_hist': time_hist}
    result_file_path = os.path.join('./results/portfolio', file_name + '.pkl')
    with open(result_file_path, 'wb') as f:
        pickle.dump(portfolio_results, f)

38208008
iter =    1, stepsize = 5.867e-01, rdiff = 4.550e-01 , f = 0.021242
Convergence achieved!
iter =    6, stepsize = 0.000e+00, rdiff = 0.000e+00,value=-3.95931
22.20832347869873
iter =    1, stepsize = 4.568e-02, rdiff = 5.461e-05 , f = 0.021242
Convergence achieved!
iter =  103, stepsize = 1.000e+00, rdiff = 3.402e-11,value=-3.95931
0.2036876678466797
syn_1000_1200_10_50_1
iter=1, stepsize=6.67e-01, criterion=6.66e-01, upper_bound=-8.22e-02, lower_bound=-5.55e+00, real_Gap=5.47e+00, f_val=-0.0822446049089623
iter=20000, stepsize=1.00e-04, criterion=7.38e-05, upper_bound=-4.98e+00, lower_bound=-4.98e+00, real_Gap=5.13e-06, f_val=-4.9838444057667655
iter=40000, stepsize=5.00e-05, criterion=3.69e-05, upper_bound=-4.98e+00, lower_bound=-4.98e+00, real_Gap=1.80e-06, f_val=-4.98384442587859
174.34367895126343
iter=1, stepsize=3.91e-01, criterion=3.90e-01, upper_bound=-8.22e-02, lower_bound=-5.55e+00, real_Gap=5.47e+00, f_val=-0.0822446049089623
iter=20000, stepsize=7.59e-05, criterio

## KL

In [4]:
policy_list = ['sc', 'line_search', 'backtracking']
files = ['a1a', 'a2a']
for file_name in files:
    kl_path = os.path.join('./data', file_name)
    kl_problem = KLProblem(path=kl_path)
    kl_results = {}
    print(file_name)
    for policy in policy_list:
        _, _, _, f_hist, time_hist = run_frank_wolfe(
                                        kl_problem,
                                        alpha_policy=policy,
                                        max_iter=50000,
                                        print_every=10000
                                        )
        kl_results[policy] = {'f_hist': f_hist, 'time_hist': time_hist}
    _, _, f_hist, time_hist = run_prox_grad(
                                        kl_problem,
                                        max_iter=50000,
                                        bb_type=3,
                                        print_every=10000
                                        )
    kl_results['prox_grad'] = {'f_hist': f_hist, 'time_hist': time_hist}
    result_file_path = os.path.join('./results/kl', file_name + '.pkl')
    with open(result_file_path, 'wb') as f:
        pickle.dump(kl_results, f)

a1a
iter=1, stepsize=3.64e-06, criterion=5.15e-06, upper_bound=-5.89e+02, lower_bound=-9.81e+04, real_Gap=9.75e+04, f_val=-588.6651794834256
iter=10000, stepsize=3.73e-05, criterion=4.99e-06, upper_bound=-1.60e+03, lower_bound=-1.61e+03, real_Gap=5.86e-01, f_val=-1604.751145623776
iter=20000, stepsize=3.28e-05, criterion=2.94e-06, upper_bound=-1.60e+03, lower_bound=-1.61e+03, real_Gap=3.42e-01, f_val=-1604.8556047801167
iter=30000, stepsize=3.14e-06, criterion=2.94e-06, upper_bound=-1.60e+03, lower_bound=-1.61e+03, real_Gap=2.12e-01, f_val=-1604.9052213473328
iter=40000, stepsize=5.36e-05, criterion=2.94e-06, upper_bound=-1.60e+03, lower_bound=-1.61e+03, real_Gap=1.38e-01, f_val=-1604.9330412087847
iter=50000, stepsize=1.19e-05, criterion=2.94e-06, upper_bound=-1.60e+03, lower_bound=-1.61e+03, real_Gap=1.08e-01, f_val=-1604.949836777469
10.905393838882446
iter=1, stepsize=2.95e-02, criterion=4.18e-02, upper_bound=-5.89e+02, lower_bound=-9.81e+04, real_Gap=9.75e+04, f_val=-588.665179483

## DWD

In [5]:
policy_list = ['standard', 'sc', 'backtracking', 'line_search']
files = ['a1a', 'a2a']
for file_name in files:
    dwd_path = os.path.join('./data', file_name)
    dwd_problem = DwdProblem(path=dwd_path)
    dwd_results= {}
    print(file_name)
    for policy in policy_list:
        _, _, _, f_hist, time_hist = run_frank_wolfe(
                                        dwd_problem,
                                        alpha_policy=policy,
                                        max_iter=50000,
                                        print_every=20000
                                        )
        dwd_results[policy] = {'f_hist': f_hist, 'time_hist': time_hist}

    _, _, f_hist, time_hist = run_prox_grad(
                                        dwd_problem,
                                        max_iter=50000,
                                        bb_type=3,
                                        print_every=20000
                                        )
    dwd_results['prox_grad'] = {'f_hist': f_hist, 'time_hist': time_hist}
    result_file_path = os.path.join('./results/dwd', file_name + '.pkl')
    with open(result_file_path, 'wb') as f:
        pickle.dump(dwd_results, f)

a1a
iter=1, stepsize=6.67e-01, criterion=1.00e+00, upper_bound=2.58e+06, lower_bound=-7.82e+10, real_Gap=7.82e+10, f_val=2576025.9999999995
iter=20000, stepsize=1.00e-04, criterion=2.63e-05, upper_bound=-4.01e+02, lower_bound=-4.01e+02, real_Gap=1.18e-05, f_val=-400.601714521745
iter=40000, stepsize=5.00e-05, criterion=1.32e-05, upper_bound=-4.01e+02, lower_bound=-4.01e+02, real_Gap=2.95e-06, f_val=-400.6017233786757
15.23496150970459
iter=1, stepsize=2.80e-06, criterion=4.20e-05, upper_bound=2.58e+06, lower_bound=-7.82e+10, real_Gap=7.82e+10, f_val=2576025.9999999995
iter=20000, stepsize=1.56e-01, criterion=4.20e-05, upper_bound=-4.01e+02, lower_bound=-4.01e+02, real_Gap=5.17e-05, f_val=-400.60137462232996
iter=40000, stepsize=3.56e-02, criterion=4.20e-05, upper_bound=-4.01e+02, lower_bound=-4.01e+02, real_Gap=-2.31e-03, f_val=-400.58843391788764
20.675881147384644
iter=1, stepsize=1.98e-05, criterion=2.97e-04, upper_bound=2.58e+06, lower_bound=-7.82e+10, real_Gap=7.82e+10, f_val=2576

## Cov Estimation

In [6]:
policy_list = ['standard', 'sc', 'backtracking', 'line_search']
files = ['cov_50.npy', 'cov_200.npy']
for file_name in files:
    cov_path = os.path.join('./data', file_name)
    cov_problem = CovEstProblem(path=cov_path)
    cov_results = {}
    print(file_name)
    x0 = cov_problem.generate_start_point()
    for policy in policy_list:
        _, _, _, f_hist, time_hist = run_frank_wolfe(
                                        cov_problem,
                                        x_0=x0,
                                        alpha_policy=policy,
                                        max_iter=50000,
                                        print_every=20000
                                        )
        cov_results[policy] = {'f_hist': f_hist, 'time_hist': time_hist}

    _, _, f_hist, time_hist = run_prox_grad(
                                        cov_problem,
                                        x_0=x0,
                                        max_iter=50000,
                                        bb_type=2,
                                        print_every=20000
                                        )
    cov_results['prox_grad'] = {'f_hist': f_hist, 'time_hist': time_hist}
    _, _, f_hist, time_hist = run_prox_newton(
                                        cov_problem,
                                        max_iter=10000,
                                        x_0=x0,
                                        use_two_phase=True,
                                        Lest='estimate',
                                        print_every=100
                                        )
    cov_results['prox_newton'] = {'f_hist': f_hist, 'time_hist': time_hist}
    result_file_path = os.path.join('./results/cov', file_name + '.pkl')
    with open(result_file_path, 'wb') as f:
        pickle.dump(cov_results, f)

cov_50.npy
iter=1, stepsize=5.00e-01, criterion=5.07e-01, upper_bound=2.12e+02, lower_bound=-8.21e+02, real_Gap=1.03e+03, f_val=212.10682745153613
iter=20000, stepsize=1.00e-04, criterion=9.90e-05, upper_bound=1.96e+02, lower_bound=1.96e+02, real_Gap=1.14e-01, f_val=196.36384790716033
iter=40000, stepsize=5.00e-05, criterion=4.95e-05, upper_bound=1.96e+02, lower_bound=1.96e+02, real_Gap=5.38e-02, f_val=196.36380517451695
13.582689046859741
iter=1, stepsize=4.51e-04, criterion=4.57e-04, upper_bound=2.12e+02, lower_bound=-8.21e+02, real_Gap=1.03e+03, f_val=212.10682745153613
Convergence achieved!
iter = 17729, stepsize = 8.020400623695373e-12, crit = 7.93850687089298e-12, upper_bound=196.36379350192615, lower_bound=196.36379348230398, real_Gap=1.9622177660494344e-08
iter=1, stepsize=1.92e-03, criterion=1.95e-03, upper_bound=2.12e+02, lower_bound=-8.21e+02, real_Gap=1.03e+03, f_val=212.10682745153613
iter=20000, stepsize=6.41e-09, criterion=4.51e-17, upper_bound=1.96e+02, lower_bound=1.96